# CS4186 Computer Vision and Image Processing

## Importing libraries

In [1]:
import os
import psnr_cal
import cv2
import numpy as np
import matplotlib.pyplot as plt

## Constant Parameters

In [2]:
# Folder Paths
imageFolder = './StereoMatchingTestings/'
predictionsFolder = './pred/'
imageStatPath = './imagestats/'

# Class of Images / Name of image folder
imagesCategory = ["Art", "Dolls", "Reindeer"]

# Images
view1 = 'view1.png'
view5 = 'view5.png'

# Feature Matcher 
matcher = "flann"

# Threshold
BEST_MATCHES_THRESHOLD = 0.7

## Loading images

In [3]:
imagesMatrix = []
for specificImage in imagesCategory:
    image1Path = imageFolder + specificImage + '/' + view1
    image2Path = imageFolder + specificImage + '/' + view5
    try:
        img1 = cv2.imread(image1Path, cv2.IMREAD_GRAYSCALE)
        img2 = cv2.imread(image2Path, cv2.IMREAD_GRAYSCALE)

        imagePair = (img1,img2)
        imagesMatrix.append(imagePair)
    except FileNotFoundError as e:
        print("Encounted error: {}".format(e))
    

In [4]:
len(imagesMatrix)

3

## SIFT Descriptor Disparity Map 

In [27]:

if not os.path.isdir(imageStatPath):
        os.mkdir(imageStatPath)

img_index = 0

for imagePair in imagesMatrix:

        siftDescriptor = cv2.SIFT_create()

        combinedCurrentPath = imageStatPath + imagesCategory[img_index]
        if not os.path.isdir(combinedCurrentPath):
                os.mkdir(combinedCurrentPath)
  

        # Extract the kepoints and descriptors
        keyPoint1, des1 = siftDescriptor.detectAndCompute(imagePair[0], None)
        keyPoint2, des2 = siftDescriptor.detectAndCompute(imagePair[1], None)

        # Display all generated keypoints
        imageShift = cv2.drawKeypoints(imagePair[0], keyPoint1, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        # if 
        # plt.imshow(imageShift)
        # plt.show()
        # cv2.imshow("SIFT_keypoints",imageShift)
        cv2.imwrite(combinedCurrentPath + "/SIFT_keypoints.png", imageShift)

        # Perform keypoint matching
        if matcher == "flann":
                params_index = dict(algorithm=1,trees=5)
                params_search = dict(checks=50)
                matches_found = cv2.FlannBasedMatcher(params_index, params_search).knnMatch(des1,des2, 2)
        elif matcher == "bf":
            matches_found = cv2.BFMatcher(cv2.NORM_L2).knnMatch(des1,des2, 2)
        else:
            print('Invalid matcher parameter set. Resorting to BruteForce matcher with NORM L2 distance measure.')
            matches_found = cv2.BFMatcher(cv2.NORM_L2).knnMatch(des1,des2, 2)

        # Filtering for best matches
        mask = [[0,0] for i in range(len(matches_found))]
        matches_feasible = []
        pointSet1 = []
        pointSet2 = []
        
        for i, (m,n) in enumerate(matches_found):
                if m.distance < BEST_MATCHES_THRESHOLD * n.distance:
                        mask[i] = [1,0]
                        matches_feasible.append([m])
                        pointSet1.append(keyPoint1[m.queryIdx].pt)
                        # print(np.array(keyPoint1[m.queryIdx].pt).shape)
                        pointSet2.append(keyPoint2[m.trainIdx].pt)
                        # print(np.array(keyPoint2[m.trainIdx].pt).shape)
        
        # print(np.array(matches_found).shape)
        # print(np.array(matches_feasible).shape)
        # print(np.array(keyPoint1).shape)
        # print(np.array(keyPoint2).shape)

        # Plotting keypoints
        params_sketch = dict(matchColor=(0, 255, 0), singlePointColor=(255,0,0),matchesMask=mask[300:500],flags=cv2.DrawMatchesFlags_DEFAULT)
        matches_feasible_keypoints = cv2.drawMatchesKnn(imagePair[0], keyPoint1, imagePair[1], keyPoint2, matches_feasible[300:500], None, **params_sketch)
        # cv2.imshow("matches_feasible", matches_feasible_keypoints)
        cv2.imwrite(combinedCurrentPath + "/matches_feasible.png", matches_feasible_keypoints)

        # Calculating Fundamental Matrix and selecting inlining values
        pointSet1 = np.int32(pointSet1)
        pointSet2 = np.int32(pointSet2)

        fm, inliers = cv2.findFundamentalMat(np.int32(pointSet1), np.int32(pointSet2), cv2.FM_RANSAC)
        
        pointSet1 = pointSet1[inliers.ravel() == 1]
        pointSet2 = pointSet2[inliers.ravel() == 1]

        firstEpiLine = cv2.computeCorrespondEpilines(pointSet2.reshape(-1, 1, 2), 2, fm).reshape(-1,3)
        secondEpiLine = cv2.computeCorrespondEpilines(pointSet1.reshape(-1, 1, 2), 2, fm).reshape(-1,3)

        height1, width1 = imagePair[0].shape
        height2, width2 = imagePair[1].shape

        _, filter1, filter2 = cv2.stereoRectifyUncalibrated(
                np.float32(pointSet1), np.float32(pointSet2), fm, imgSize=(width1, height1)
        )

        # Rectify the images
        img1_fixed = cv2.warpPerspective(imagePair[0], filter1, (width1, height1))
        cv2.imwrite(combinedCurrentPath + "/view1_rectified.png", img1_fixed)

        img2_fixed = cv2.warpPerspective(imagePair[1], filter2, (width2, height2))
        cv2.imwrite(combinedCurrentPath + "/view2_rectified.png", img2_fixed)

        # Creating the stero object

        windowSize = 3
        bSize = 20
        minDisp = -128
        maxDisp = 128
        nDisp = maxDisp - minDisp
        disp12MaxDiff = 0
        uRatio = 2
        speckleWindowSize = 200
        speckleRange = 2

        stereo = cv2.StereoSGBM_create(
                minDisparity=minDisp,
                numDisparities=nDisp,
                blockSize=bSize,
                uniquenessRatio=uRatio,
                speckleWindowSize=speckleWindowSize,
                speckleRange=speckleRange,
                disp12MaxDiff=disp12MaxDiff,
                P1=8 * 1 * windowSize * 2,
                P2=32 * 1 * windowSize * 2,
        )  

        disparity_SGBM = stereo.compute(img1_fixed, img2_fixed)

        disparity_map = cv2.ximgproc.createRightMatcher(stereo).compute(img2_fixed,img1_fixed)

        wls_filter = cv2.ximgproc.createDisparityWLSFilter(matcher_left=stereo)
        wls_filter.setLambda(35000)
        wls_filter.setSigmaColor(1.5)
        filtered_disp = wls_filter.filter(disparity_SGBM, img1_fixed, disparity_map_right=disparity_map)
        filtered_disp = cv2.normalize(filtered_disp, filtered_disp, alpha=255, beta=0, norm_type=cv2.NORM_MINMAX)
        cv2.imwrite(predictionsFolder + imagesCategory[img_index] + "/disp1.png", filtered_disp)
        img_index+=1

In [28]:
psnr_cal.test()

The Peak-SNR value is %0.4f 
 5.334773826753589
The Peak-SNR value is %0.4f 
 10.322574474920144
The Peak-SNR value is %0.4f 
 12.580840080062183
